In [1]:
import json
import requests
import numpy as np
from dateutil.relativedelta import relativedelta
from config import lw_key
import pandas as pd
from datetime import date, timedelta, datetime
import matplotlib.pyplot as plt
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from os import path


In [2]:
url= "https://api.worldweatheronline.com/premium/v1/weather.ashx?"
past_url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx?"
# query_url =f"{url}appid={api_key}&zip="

In [3]:
zipcode = input("Enter 5-digit Zipcode:")

Enter 5-digit Zipcode:21224


In [8]:
# zip_df = nomi.query_postal_code(zipcode)


In [9]:
# pandas apply pgeocode to ZIPCODE1
# md_zips = df['ZIPCODE1'].apply(nomi.query_postal_code)
# md_zips.head()

In [10]:
response = requests.get(f"{url}key={lw_key}&q={zipcode}&num_of_days=8&tp=24&cc=no&mca=no&aqi=yes&format=json").json()
response = response['data']


In [11]:
weather_dict ={
    "Dates":[],
    "Cloudcover":[],
    "Humidity":[],
    "PrecipInch":[],
    "Pressure":[],
    "FeelsLike":[],
    "HeatIndex":[],
    "MaxTemp":[],
    "MinTemp":[],
    "SunHours":[],
    "UVIndex":[],
}

weather_dict["Dates"]= ([response["weather"][i]["date"] for i in range(len(response["weather"]))])
weather_dict["Cloudcover"]=([response["weather"][i]["hourly"][0]["cloudcover"] for i in range(len(response["weather"]))])
weather_dict["Humidity"]= ([response["weather"][i]["hourly"][0]["humidity"] for i in range(len(response["weather"]))])
weather_dict["PrecipInch"]= ([response["weather"][i]["hourly"][0]["precipInches"] for i in range(len(response["weather"]))])
weather_dict["Pressure"]= ([response["weather"][i]["hourly"][0]["pressure"] for i in range(len(response["weather"]))])
weather_dict["FeelsLike"]= ([response["weather"][i]["hourly"][0]["FeelsLikeF"] for i in range(len(response["weather"]))])
weather_dict["HeatIndex"]= ([response["weather"][i]["hourly"][0]["HeatIndexF"] for i in range(len(response["weather"]))])
weather_dict["MaxTemp"]= ([response["weather"][i]["maxtempF"] for i in range(len(response["weather"]))])
weather_dict["MinTemp"]=([response["weather"][i]["mintempF"] for i in range(len(response["weather"]))])
weather_dict["SunHours"]=([response["weather"][i]["sunHour"] for i in range(len(response["weather"]))])
weather_dict["UVIndex"]= ([response["weather"][i]["hourly"][0]["uvIndex"] for i in range(len(response["weather"]))])
weather_df= pd.DataFrame.from_dict(weather_dict,orient='index').transpose()
weather_df= weather_df.apply(pd.to_numeric, errors= 'ignore')
weather_df["TempDelta"] = weather_df.MaxTemp - weather_df.MinTemp
weather_df["BarChange"] =weather_df["Pressure"].pct_change()
weather_df["HeatChange"] =weather_df["HeatIndex"].pct_change()
weather_df["HumChange"] =weather_df["Humidity"].pct_change()
weather_df= weather_df.iloc[1:]
weather_df.head(10)


,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange
1,2021-04-17,82,63,0.0,1011,45,49,52,46,6.6,3,6,0.002976,0.000000,0.067797
2,2021-04-18,49,61,0.0,1013,49,51,56,45,10.4,4,11,0.001978,0.040816,-0.031746
3,2021-04-19,61,70,0.0,1015,50,51,56,48,7.9,4,8,0.001974,0.000000,0.147541
4,2021-04-20,29,70,0.0,1017,54,55,62,48,10.4,1,14,0.001970,0.078431,0.000000
5,2021-04-21,41,59,0.0,1012,46,51,54,44,10.4,3,10,-0.004916,-0.072727,-0.157143
6,2021-04-22,17,48,0.0,1021,42,48,57,39,11.6,3,18,0.008893,-0.058824,-0.186441
7,2021-04-23,17,47,0.0,1023,52,54,64,44,11.6,4,20,0.001959,0.125000,-0.020833


In [13]:
dates= input("Dates you had a migraine (YYYY-MM-DD):")

Dates you had a migraine (YYYY-MM-DD):2020-10-15,2020-10-20,2020-10-26,2020-10-28,2020-10-31,2020-11-02,2020-11-06,2020-11-18,2020-11-21,2020-11-24,2020-12-01,2020-12-07,2020-12-16,2020-12-24,2020-12-27,2021-01-01,2021-01-04,2021-01-10,2021-02-05,2021-02-06,2021-02-07,2021-02-08,2021-02-26 2021-02-27,2021-02-28,2021-03-01,2021-03-02,2021-03-03,2021-03-20,2021-04-05,2021-04-08,2021-04-10


In [14]:
dates=dates.split(",")
dates= [i.strip() for i in dates]
dates_df = pd.DataFrame(dates,columns=['Dates'])
dates_df["Migraine"] = 1

In [15]:
history_dict ={
    "Dates":[],
    "Cloudcover":[],
    "Humidity":[],
    "PrecipInch":[],
    "Pressure":[],
    "FeelsLike":[],
    "HeatIndex":[],
    "MaxTemp":[],
    "MinTemp":[],
    "SunHours":[],
    "UVIndex":[],
    }

In [16]:
for i in range(6,0,-1):
    date= (datetime.today()-relativedelta(months=i)).strftime('%Y-%m-%d')
    enddate=(datetime.today()-relativedelta(months=i-1)).strftime('%Y-%m-%d')
    responses = requests.get(f"{past_url}key={lw_key}&q={zipcode}&date={date}&enddate={enddate}&tp=24&mca=no&aqi=yes&format=json").json()
    responses = responses['data']
    history_dict["Dates"].append([responses["weather"][i]["date"] for i in range(len(responses["weather"]))])
    history_dict["Cloudcover"].append([responses["weather"][i]["hourly"][0]["cloudcover"] for i in range(len(responses["weather"]))])
    history_dict["Humidity"].append([responses["weather"][i]["hourly"][0]["humidity"] for i in range(len(responses["weather"]))])
    history_dict["PrecipInch"].append([responses["weather"][i]["hourly"][0]["precipInches"] for i in range(len(responses["weather"]))])
    history_dict["Pressure"].append([responses["weather"][i]["hourly"][0]["pressure"] for i in range(len(responses["weather"]))])
    history_dict["FeelsLike"].append([responses["weather"][i]["hourly"][0]["FeelsLikeF"] for i in range(len(responses["weather"]))])
    history_dict["HeatIndex"].append([responses["weather"][i]["hourly"][0]["HeatIndexF"] for i in range(len(responses["weather"]))])
    history_dict["MaxTemp"].append([responses["weather"][i]["maxtempF"] for i in range(len(responses["weather"]))])
    history_dict["MinTemp"].append([responses["weather"][i]["mintempF"] for i in range(len(responses["weather"]))])
    history_dict["SunHours"].append([responses["weather"][i]["sunHour"] for i in range(len(responses["weather"]))])
    history_dict["UVIndex"].append([responses["weather"][i]["hourly"][0]["uvIndex"] for i in range(len(responses["weather"]))])

In [17]:
history_df= pd.DataFrame.from_dict(history_dict,orient='index').transpose()
history_df= history_df.apply(pd.to_numeric, errors= 'ignore')


In [18]:
history_df= history_df.apply(pd.Series.explode).reset_index(drop=True)
history_df= history_df.apply(pd.to_numeric, errors= 'ignore')
history_df.reset_index(drop=True)
history_df.head()

,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex
0,2020-10-16,78,60,0.0,1022,56,58,58,54,7.1,3
1,2020-10-17,18,47,0.0,1026,50,52,59,40,11.0,4
2,2020-10-18,1,51,0.0,1029,57,58,62,55,11.0,4
3,2020-10-19,30,77,0.0,1026,61,61,66,57,11.0,5
4,2020-10-20,25,75,0.0,1024,65,66,73,59,11.0,5


In [19]:
history_df["TempDelta"]= history_df.MaxTemp - weather_df.MinTemp
history_df["BarChange"]= history_df["Pressure"].pct_change()
history_df["HeatChange"]= history_df["HeatIndex"].pct_change()
history_df["HumChange"]= history_df["Humidity"].pct_change()
history_df= pd.merge(history_df,dates_df, on = 'Dates', how = 'left')
history_df= history_df.fillna(0)
history_df= history_df.iloc[1:]
history_df.head(100)

,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange,Migraine
1,2020-10-17,18,47,0.0,1026,50,52,59,40,11.0,4,13.0,0.003914,-0.103448,-0.216667,0.0
2,2020-10-18,1,51,0.0,1029,57,58,62,55,11.0,4,17.0,0.002924,0.115385,0.085106,0.0
3,2020-10-19,30,77,0.0,1026,61,61,66,57,11.0,5,18.0,-0.002915,0.051724,0.509804,0.0
4,2020-10-20,25,75,0.0,1024,65,66,73,59,11.0,5,25.0,-0.001949,0.081967,-0.025974,1.0
5,2020-10-21,54,84,0.0,1024,69,72,75,65,8.5,5,31.0,0.000000,0.090909,0.120000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2021-01-17,28,58,0.0,1006,34,40,45,36,9.9,2,0.0,0.004995,-0.024390,-0.121212,0.0
97,2021-01-18,36,56,0.0,1009,33,39,42,36,9.9,2,0.0,0.002982,-0.025000,-0.034483,0.0
98,2021-01-19,34,55,0.0,1019,32,38,44,34,9.9,1,0.0,0.009911,-0.025641,-0.017857,0.0
99,2021-01-20,28,50,0.0,1019,28,36,37,34,9.9,1,0.0,0.000000,-0.052632,-0.090909,0.0


In [20]:
#history_df.to_csv("../../Desktop/test.csv")

In [21]:
#df_forecast["weather_df"].value_counts()

In [22]:
history_df.columns

Index(['Dates', 'Cloudcover', 'Humidity', 'PrecipInch', 'Pressure',
       'FeelsLike', 'HeatIndex', 'MaxTemp', 'MinTemp', 'SunHours', 'UVIndex',
       'TempDelta', 'BarChange', 'HeatChange', 'HumChange', 'Migraine'],
      dtype='object')

In [23]:
history_df.head()

,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange,Migraine
1,2020-10-17,18,47,0.0,1026,50,52,59,40,11.0,4,13.0,0.003914,-0.103448,-0.216667,0.0
2,2020-10-18,1,51,0.0,1029,57,58,62,55,11.0,4,17.0,0.002924,0.115385,0.085106,0.0
3,2020-10-19,30,77,0.0,1026,61,61,66,57,11.0,5,18.0,-0.002915,0.051724,0.509804,0.0
4,2020-10-20,25,75,0.0,1024,65,66,73,59,11.0,5,25.0,-0.001949,0.081967,-0.025974,1.0
5,2020-10-21,54,84,0.0,1024,69,72,75,65,8.5,5,31.0,0.000000,0.090909,0.120000,0.0


In [66]:
migraine_df=history_df.drop("Dates",axis=1)

In [67]:
migraine_df.columns

Index(['Cloudcover', 'Humidity', 'PrecipInch', 'Pressure', 'FeelsLike',
       'HeatIndex', 'MaxTemp', 'MinTemp', 'SunHours', 'UVIndex', 'TempDelta',
       'BarChange', 'HeatChange', 'HumChange', 'Migraine'],
      dtype='object')

In [68]:
# Assign X (data) and y (target)
X = migraine_df.drop("Migraine", axis=1)
y = migraine_df["Migraine"]
print(X.shape, y.shape)

(187, 14) (187,)


In [69]:
#Split our data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [70]:
#Fit on the training data
X_scaler = StandardScaler().fit(X_train)
X_scaler.get_params()

{'copy': True, 'with_mean': True, 'with_std': True}

In [71]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_test_scaled

array([[ 6.71309743e-01,  4.01073697e-01,  1.63532020e-01,
        -1.51091847e+00,  1.08119204e+00,  9.66486038e-01,
         1.08717344e+00,  1.14378932e+00, -6.67544338e-02,
         5.72023412e-01, -2.03004669e-01, -1.23295713e+00,
         2.86484623e-01, -7.50639507e-01],
       [ 1.04761951e+00,  9.38397310e-01,  1.36850480e+00,
         1.31125453e+00, -7.64745591e-01, -8.59403778e-01,
        -1.10676215e+00, -1.04456577e+00, -1.79891690e+00,
        -1.13187611e+00, -2.03004669e-01,  1.00517836e+00,
        -1.63964589e+00,  1.76248191e-01],
       [-1.24159823e+00, -1.36249916e-01, -4.38954370e-01,
        -3.26373749e-02,  8.13091577e-02,  1.01590862e-01,
         8.16196276e-02,  1.92330587e-01,  2.95791198e-01,
         5.72023412e-01, -2.03004669e-01,  1.01546674e+00,
         9.72408799e-01,  1.59997923e-01],
       [ 1.14169695e+00,  1.47572092e+00,  1.63532020e-01,
         3.70530198e-01,  1.58223226e-01,  1.01590862e-01,
         2.64447593e-01,  2.03884015e-03, -3.

In [72]:
y_train.value_counts()

0.0    115
1.0     25
Name: Migraine, dtype: int64

In [80]:
y_test.value_counts()

0.0    42
1.0     5
Name: Migraine, dtype: int64

In [82]:
# X_test_scaled.value_counts()

In [53]:
# Create the SVC Model
model = SVC(kernel="linear")
model

SVC(kernel='linear')

In [54]:
model.fit(X_train, y_train)

SVC(kernel='linear')

In [55]:
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.894


In [56]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=["candidate", "false positive"]))

                precision    recall  f1-score   support

     candidate       0.89      1.00      0.94        42
false positive       0.00      0.00      0.00         5

      accuracy                           0.89        47
     macro avg       0.45      0.50      0.47        47
  weighted avg       0.80      0.89      0.84        47



C:\Users\kdodd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [57]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [58]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.821, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.821, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.821, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.821, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.821, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.821, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ...........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] .................... C=5, gamma=0.001, score=0.821, total=   0.0s
[CV] C=5, gamma=0.005 ................................................
[CV] .................... C=5, gamma=0.005, score=0.821, total=   0.0s
[CV] C=5, gamma=0.005 ................................................
[CV] .................... C=5, gamma=0.005, score=0.821, total=   0.0s
[CV] C=5, gamma=0.005 ................................................
[CV] .................... C=5, gamma=0.005, score=0.821, total=   0.0s
[CV] C=5, gamma=0.005 ................................................
[CV] .................... C=5, gamma=0.005, score=0.821, total=   0.0s
[CV] C=5, gamma=0.005 ................................................
[CV] .................... C=5, gamma=0.005, score=0.821, total=   0.0s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .................. C=10, gamma=0.0001, score=0.821, total=   0.0s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.4s finished


GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10, 50],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             verbose=3)

In [59]:
# List the best parameters for this dataset
print(grid.best_params_)

{'C': 1, 'gamma': 0.0001}


In [60]:
# List the best score
print(grid.best_score_)

0.8214285714285714


In [84]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [85]:
print('Test Acc: %.3f' % grid.score(X_test_scaled, y_test))

Test Acc: 0.894


In [86]:
predictions

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [87]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["candidate", "false positive"]))

                precision    recall  f1-score   support

     candidate       0.89      1.00      0.94        42
false positive       0.00      0.00      0.00         5

      accuracy                           0.89        47
     macro avg       0.45      0.50      0.47        47
  weighted avg       0.80      0.89      0.84        47



C:\Users\kdodd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
